## Testing on Temporary input token embeddings converting to the context vectors

Simplified Self attention

In [2]:
import torch

# Temp input embeddings

In [13]:
inputs = torch.tensor([[1.5,1.4,1.3], # Embeddings for journey
                       [2.5,2.4,2.3], # Embeddings for starts
                       [3.5,3.4,3.3], # Embeddings for in
                       [4.5,4.4,4.3]]) # Embeddings for the

In [6]:
inputs.shape

torch.Size([4, 3])

In [7]:
query_vector = torch.empty(inputs.shape[0])

In [9]:
query_vector.shape

torch.Size([4])

In [10]:
for i,inpt in enumerate(inputs):
    query_vector[i] = torch.dot(inputs[0],inpt)

In [11]:
query_vector

tensor([ 5.9000, 10.1000, 14.3000, 18.5000])

In [15]:
qqq = query_vector/query_vector.sum()

In [16]:
qqq.sum()

tensor(1.)

In [17]:
def softmax_naive(x):
    attention_weights = torch.exp(x)/ torch.exp(x).sum(dim=0)
    return attention_weights

final_weights = softmax_naive(qqq)

In [18]:
final_weights

tensor([0.2187, 0.2384, 0.2598, 0.2831])

In [19]:
final_weights.sum()

tensor(1.)

In [9]:
inputs.shape

torch.Size([3, 3])

In [24]:
query = inputs[1]
context_vector2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vector2 += final_weights[i]*x_i
    print(context_vector2)
    
    

tensor([0.3281, 0.3062, 0.2843])
tensor([0.9240, 0.8783, 0.8326])
tensor([1.8332, 1.7615, 1.6899])
tensor([3.1074, 3.0074, 2.9074])


In [25]:
context_vector2

tensor([3.1074, 3.0074, 2.9074])

In [20]:
attn_scores = torch.empty(inputs.shape)

In [21]:
attn_scores.shape


torch.Size([3, 3])

In [27]:
for i,x_i in enumerate(inputs):
    for j,x_j in enumerate(inputs):
        print(x_i)
        print(x_j)
        
        attn_scores[i,j] = torch.dot(x_i,x_j)
    attn_scores

tensor([1.5000, 1.4000, 1.3000])
tensor([1.5000, 1.4000, 1.3000])
tensor([1.5000, 1.4000, 1.3000])
tensor([2.5000, 2.4000, 2.3000])
tensor([1.5000, 1.4000, 1.3000])
tensor([3.5000, 3.4000, 3.3000])
tensor([2.5000, 2.4000, 2.3000])
tensor([1.5000, 1.4000, 1.3000])
tensor([2.5000, 2.4000, 2.3000])
tensor([2.5000, 2.4000, 2.3000])
tensor([2.5000, 2.4000, 2.3000])
tensor([3.5000, 3.4000, 3.3000])
tensor([3.5000, 3.4000, 3.3000])
tensor([1.5000, 1.4000, 1.3000])
tensor([3.5000, 3.4000, 3.3000])
tensor([2.5000, 2.4000, 2.3000])
tensor([3.5000, 3.4000, 3.3000])
tensor([3.5000, 3.4000, 3.3000])


In [23]:
attn_scores

tensor([[ 5.9000, 10.1000, 14.3000],
        [10.1000, 17.3000, 24.5000],
        [14.3000, 24.5000, 34.7000]])

In [24]:
attn_weights = torch.softmax(attn_scores,dim=1)

In [26]:
attn_weights.sum(dim=1)

tensor([1., 1., 1.])

In [30]:
attn_scores_final = inputs @ inputs.T

In [31]:
attn_scores_final

tensor([[ 5.9000, 10.1000, 14.3000, 18.5000],
        [10.1000, 17.3000, 24.5000, 31.7000],
        [14.3000, 24.5000, 34.7000, 44.9000],
        [18.5000, 31.7000, 44.9000, 58.1000]])

In [38]:
attn_weights_final = torch.softmax(attn_scores_final,dim=-1)

In [37]:
attn_weights_final

tensor([[3.3214e-06, 4.1583e-10, 5.1354e-14, 6.3378e-18],
        [2.2150e-04, 5.5697e-07, 1.3816e-09, 3.4247e-12],
        [1.4771e-02, 7.4603e-04, 3.7169e-05, 1.8506e-06],
        [9.8500e-01, 9.9925e-01, 9.9996e-01, 1.0000e+00]])

In [39]:
attn_weights_final

tensor([[3.3214e-06, 2.2150e-04, 1.4771e-02, 9.8500e-01],
        [4.1583e-10, 5.5697e-07, 7.4603e-04, 9.9925e-01],
        [5.1354e-14, 1.3816e-09, 3.7169e-05, 9.9996e-01],
        [6.3378e-18, 3.4247e-12, 1.8506e-06, 1.0000e+00]])

In [42]:
contex_vector_final = attn_weights_final @ inputs

In [43]:
contex_vector_final

tensor([[4.4848, 4.3848, 4.2848],
        [4.4993, 4.3993, 4.2993],
        [4.5000, 4.4000, 4.3000],
        [4.5000, 4.4000, 4.3000]])

## Self Attention with  Trainable Weights

In [5]:
input_dim = inputs.shape[1]
output_dim = 2 

In [24]:
torch.manual_seed(42)

In [33]:
Query_w = torch.nn.Parameter(torch.randn(input_dim,output_dim), requires_grad=False)
Key_w = torch.nn.Parameter(torch.randn(input_dim,output_dim), requires_grad=False)
Value_w = torch.nn.Parameter(torch.randn(input_dim,output_dim), requires_grad=False)

In [34]:
Query_w

Parameter containing:
tensor([[-0.1335,  0.3415],
        [-0.0716, -0.0909],
        [-1.3297, -0.5426]])

In [35]:
x_2 = inputs[1]

In [36]:
Query_2 = x_2 @ Query_w 
Key_2 = x_2 @ Key_w
Value_2 = x_2 @ Value_w 

In [37]:
Query_2

tensor([-3.5638, -0.6124])

## Now implement the whole Input * Q,K and V Embeddings to obtain the Q,K and V metrics

In [38]:
Query = inputs @ Query_w
Key = inputs @ Key_w
Value = inputs @ Value_w


In [39]:
Query

tensor([[-2.0290, -0.3204],
        [-3.5638, -0.6124],
        [-5.0985, -0.9043],
        [-6.6333, -1.1963]])

## Finding the Attention Score For Self Attention using Q,K,V


#Attention score for Query 2 with all the key embeddings

In [40]:
Query_2

tensor([-3.5638, -0.6124])

In [41]:
attention_scores2 = Query_2 @ Key.T

In [42]:
attention_scores2

tensor([2.8482, 5.1403, 7.4324, 9.7245])

In [43]:
d_k = Key.shape[-1]

In [44]:
attention_weight2 = torch.softmax(attention_scores2/ d_k**0.5,dim=-1)

## As seen above we used SQRT of dim 2 which will be d_k**0.5 division with attn_score2 reason is that 
1.to minimize the variance when the input embed are high it will produce the high variance while obtaining the dot product so the normal softmax wont be able to normalize that scores near to one.

2. Because the highest scores will get the highest probabilites which will make an llm peaky and will focus on that particular query more.


In [45]:
attention_weight2

tensor([0.0062, 0.0314, 0.1589, 0.8035])

In [46]:
context_vector2 = attention_weight2 @ Value

In [48]:
context_vector2.T

C:\Users\DEV\AppData\Local\Temp\ipykernel_11192\152547829.py:1: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:3687.)
  context_vector2.T


tensor([11.5268, -0.5776])

In [49]:
context_vector2


tensor([11.5268, -0.5776])

## Creating a Self attention Class

In [90]:
import torch.nn as nn
class SelfAttentionV1(nn.Module):
    def __init__(self,input_dim,output_dim):
        super().__init__()
        self.Query_w = torch.nn.Parameter(torch.randn(input_dim,output_dim), requires_grad=True)
        self.Key_w = torch.nn.Parameter(torch.randn(input_dim,output_dim), requires_grad=True)
        self.Value_w = torch.nn.Parameter(torch.randn(input_dim,output_dim), requires_grad=True)
        
    def forward(self,inputs):
        Query = inputs @ self.Query_w
        Key = inputs @ self.Key_w
        Value = inputs @ self.Value_w
        attention_scores = Query @ Key.T
        d_k = Key.shape[-1]
        attention_weight = torch.softmax(attention_scores/ d_k**0.5,dim=-1)
        context_vector = attention_weight @ Value
        return context_vector

In [91]:
attention = SelfAttentionV1(input_dim,output_dim)

In [92]:

context_vec = attention(inputs)

TypeError: 'Parameter' object is not callable

In [79]:
context_vec

tensor([[4.2190, 0.9576],
        [4.3600, 0.9861],
        [4.4031, 0.9949],
        [4.4175, 0.9978]], grad_fn=<MmBackward0>)

In [102]:
import torch.nn as nn
class SelfAttentionV2(nn.Module):
    def __init__(self, input_dim, output_dim,qkv_bias=False):
        super().__init__()
        self.Query_w = nn.Linear(input_dim, output_dim, bias=False)
        self.Key_w = nn.Linear(input_dim, output_dim, bias=False)
        self.Value_w = nn.Linear(input_dim, output_dim, bias=False)
        
        
    def forward(self,inputs):
        Key = self.Key_w(inputs)
        Query = self.Query_w(inputs)
        Value = self.Value_w(inputs)
        attention_scores = Query @ Key.T
        d_k = Key.shape[-1]
        attention_weight = torch.softmax(attention_scores/ d_k**0.5,dim=-1)
        context_vector = attention_weight @ Value
        return context_vector        
        

In [103]:
inputs

tensor([[1.5000, 1.4000, 1.3000],
        [2.5000, 2.4000, 2.3000],
        [3.5000, 3.4000, 3.3000],
        [4.5000, 4.4000, 4.3000]])

In [104]:
torch.manual_seed(789)
attentionv2 = SelfAttentionV2(input_dim,output_dim)
attentionv2(inputs)

tensor([[-0.2558,  0.8838],
        [-0.2832,  0.9576],
        [-0.3007,  1.0049],
        [-0.3114,  1.0338]], grad_fn=<MmBackward0>)